In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Define the API key and search engine ID
API_KEY = ""  # Replace with your actual CSE ID
SEARCH_ENGINE_ID = ""  # Replace with your actual CSE ID

# Function to extract phone numbers and text from a given URL
def extract_phone_numbers_and_text(url):
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
        print(f"Connection Successful for {url}")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return set(), ""

    soup = BeautifulSoup(html_content, 'html.parser')
    text_content = soup.get_text(separator=' ', strip=True)

    patterns = [
        re.compile(r'\+44\s?7\d{3}\s?\d{3}\s?\d{3}'),               # Mobile numbers with country code
        re.compile(r'\+44\s?2\d{3}\s?\d{3}\s?\d{4}'),               # Landline numbers with country code
        re.compile(r'07\d{3}\s?\d{3}\s?\d{3}'),                     # Mobile numbers without country code
        re.compile(r'0\d{3}\s?\d{3}\s?\d{4}'),                      # Landline numbers like 0123 456 7890
        re.compile(r'0\d{4}\s?\d{3}\s?\d{3}'),                      # Landline numbers like 01234 567 890
        re.compile(r'0330\s?\d{3}\s?\d{4}'),                        # Special numbers like 0330 XXX XXXX
        re.compile(r'0207\s?\d{3}\s?\d{4}'),                        # London numbers like 0207 XXX XXXX
        re.compile(r'\+44\s?\(?0\)?\s?\d{3,4}\s?\d{3}\s?\d{4}'),    # Landline numbers like +44 XXX XXX XXXX
        re.compile(r'0208\s?\d{3}\s?\d{4}')                         # London numbers like 0208 XXX XXXX
    ]

    matches = set()  # Use a set to store unique phone numbers
    for pattern in patterns:
        matches.update(pattern.findall(text_content))

    return matches, text_content

# Function to generate URLs and extract data
def process_urls(base_urls):
    all_phone_numbers = set()
    all_texts = []

    for base_url in base_urls:
        urls_to_check = [
            base_url,
            base_url.rstrip('/') + '/contact-us/',
            base_url.rstrip('/') + '/contact/'
        ]
        
        for url in urls_to_check:
            phone_numbers, text = extract_phone_numbers_and_text(url)
            all_phone_numbers.update(phone_numbers)
            all_texts.append({'url': url, 'text': text})

    # Convert to DataFrame
    df_texts = pd.DataFrame(all_texts)

    # Display the extracted text content
    # print("Extracted Text Content:")
    # display(df_texts)

    # Print unique phone numbers
    print("\nUnique Phone Numbers:")
    print(all_phone_numbers)

    # Optionally, display the unique phone numbers using pandas
    df_phone_numbers = pd.DataFrame(list(all_phone_numbers), columns=['Phone Numbers'])
    # display(df_phone_numbers)

# Load company names from Excel file
file_path = 'yourfile.xlsx'  # Replace with the path to your Excel file
df_companies = pd.read_excel(file_path)

# Iterate over each company name and process
for company_name in df_companies['CompanyName']:  # Replace 'CompanyName' with your actual column name
    query = company_name
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={API_KEY}&cx={SEARCH_ENGINE_ID}"

    # Make the request to the Google Custom Search API
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        search_results = response.json()
        
        # Extract the first link from the search results
        first_result = search_results.get("items", [])[0]
        first_link = first_result.get("link") if first_result else None
        
        # Store the first link in the base_urls variable
        base_urls = [first_link] if first_link else []
        
        # Print the base_urls variable
        print(f"base_urls for {company_name}: {base_urls}")
        
        # Process the URLs
        if base_urls:
            process_urls(base_urls)
    else:
        print(f"Error: {response.status_code}")


FileNotFoundError: [Errno 2] No such file or directory: 'company_names.xlsx'